Import libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
from pandas.io.json import json_normalize
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
# used links:
#https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722
#https://www.datacamp.com/community/tutorials/web-scraping-using-python

Use beautiful soup to request the table from Wikipedia page and create a dataframe

In [2]:
url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = urlopen(url)
#print(soup.prettify())
soup = BeautifulSoup(html, 'lxml')
Wikitable = soup.find('table',{'class':'wikitable sortable'})
rows = Wikitable.find_all('tr')
list_rows = []
for row in rows:
    cells = row.find_all('td')
    to_clean = str(cells)
    what = re.compile('<.*?>')
    cleaned = (re.sub(what, '',to_clean))
    list_rows.append(cleaned)
dataframe = pd.DataFrame(list_rows)

Data preparation step

In [3]:
table1 = dataframe[0].str.split(',', expand=True)
table1[2] = table1[2].str.strip('\n]')
table1[0] = table1[0].str.strip('[')
table1 = table1.iloc[1:]
table1.columns = ['Postcode','Borough', 'Neighborhood']
table1['Postcode'] = table1['Postcode'].str.strip()
table1['Borough'] = table1['Borough'].str.strip()
table1['Neighborhood'] = table1['Neighborhood'].str.strip()
table2 = table1[table1['Borough'] != 'Not assigned']
table3 = table2.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))
table3['Neighborhood'] = np.where(table3['Neighborhood'] == 'Not assigned', table3['Borough'], table3['Neighborhood'])

Call df shape

In [4]:
table3.shape 

(103, 3)

Save df as csv

In [5]:
table3.to_csv('data.csv', index=False)